In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import nltk
from nltk.chunk import ne_chunk
from nltk.tag import pos_tag
from collections import Counter

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
combine_df = pd.read_pickle('/content/drive/My Drive/Нейронные сети/Neural Language Processing/Lesson 1/combine_df.zip', compression="zip")
combine_df.head()

,id,label,tweet,processed_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [5]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

### проверка вытаскивания NER с помощью nltk

In [6]:
import requests
from bs4 import BeautifulSoup
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

document = "oh Kate Pirson is the donald having bad campaign day never mind his hair how many dead people does he need to boost his"

nltk.pos_tag(nltk.word_tokenize(document))

[('oh', 'JJ'),
 ('Kate', 'NNP'),
 ('Pirson', 'NNP'),
 ('is', 'VBZ'),
 ('the', 'DT'),
 ('donald', 'NN'),
 ('having', 'VBG'),
 ('bad', 'JJ'),
 ('campaign', 'NN'),
 ('day', 'NN'),
 ('never', 'RB'),
 ('mind', 'VB'),
 ('his', 'PRP$'),
 ('hair', 'NN'),
 ('how', 'WRB'),
 ('many', 'JJ'),
 ('dead', 'JJ'),
 ('people', 'NNS'),
 ('does', 'VBZ'),
 ('he', 'PRP'),
 ('need', 'VB'),
 ('to', 'TO'),
 ('boost', 'VB'),
 ('his', 'PRP$')]

In [7]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Kate Pirson', 'ORGANIZATION')}

In [8]:
text = "oh dear is the donald having bad campaign day never mind his hair how many dead people does he need to boost his"

In [9]:
# nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text)))

In [10]:
ner_list = [(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') ]
ner_list

[]

In [11]:
for entity in ner_list:
    print(entity)

### выполнение задания

In [12]:
cntr = Counter()
ents_dict = {}

In [13]:
%%time
for i in range(len(combine_df)):
    text = combine_df.iloc[i]["processed_tweet"]
    # print(text)
    ner_list = [(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') ]
    for entity in ner_list:
        cntr[entity[0]] += 1
        if entity[0] not in ents_dict:
            ents_dict[entity[0]] = entity[1]
        else:
            if ents_dict[entity[0]] != entity[1]:
                print("ENTITIES MISMATCH:\n\tentity text: %s\n\tdict label: %s\n\tnew label: %s\n" % (entity[0], ents_dict[entity[0]], entity[1]))


CPU times: user 3min 23s, sys: 703 ms, total: 3min 23s
Wall time: 3min 23s


In [14]:
cntr.most_common(20)

[]

In [15]:
%%time
tmp_cntr = 0
popular_orgs_and_persons = []
for i, common in enumerate(cntr.most_common(10000)):
    word = common[0]
    count = common[1]
    ent_label = ents_dict[word]
    if ent_label == "PERSON" or ent_label == "ORG":
        popular_orgs_and_persons.append((word, ent_label, count))
    else:
        print(i, word, ent_label, count)
    if len(popular_orgs_and_persons) == 20:
        break

CPU times: user 15 µs, sys: 1 µs, total: 16 µs
Wall time: 18.1 µs


Попытка использования nltk на обработанно датасете дала ничего. Поэтому я попробовал использовать его на необработанных твитах.

In [19]:
%%time
for i in range(len(combine_df)):
    text = combine_df.iloc[i]["tweet"]
    # print(text)
    ner_list = [(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') ]
    for entity in ner_list:
        cntr[entity[0]] += 1
        if entity[0] not in ents_dict:
            ents_dict[entity[0]] = entity[1]
        else:
            if ents_dict[entity[0]] != entity[1]:
                print("ENTITIES MISMATCH:\n\tentity text: %s\n\tdict label: %s\n\tnew label: %s\n" % (entity[0], ents_dict[entity[0]], entity[1]))


ENTITIES MISMATCH:
	entity text: l.a.
	dict label: GPE
	new label: ORGANIZATION

CPU times: user 4min 30s, sys: 832 ms, total: 4min 31s
Wall time: 4min 31s


In [20]:
cntr.most_common(20)

[('u.s.', 19),
 ('mr.', 4),
 ('en-ger-land', 2),
 ('atÂ', 2),
 ('u.s', 2),
 ('thatÂ', 2),
 ('kÃ²', 2),
 ('euro2016Â', 2),
 ('rainingÂ', 2),
 ('rainÂ', 2),
 ('weatherÂ', 2),
 ('lightningÂ', 2),
 ('l.a.', 2),
 ('felicitÃ', 2),
 ('zÃ¼rich', 2),
 ('fÃªte', 2),
 ('brisk_and_vagabond-eyeopener_2007__enviromental_product-', 1),
 ('nycÂ', 1),
 ('travelÂ', 1),
 ('mr. velicaria', 1)]

In [21]:
%%time
tmp_cntr = 0
popular_orgs_and_persons = []
for i, common in enumerate(cntr.most_common(10000)):
    word = common[0]
    count = common[1]
    ent_label = ents_dict[word]
    if ent_label == "PERSON" or ent_label == "ORG":
        popular_orgs_and_persons.append((word, ent_label, count))
    else:
        print(i, word, ent_label, count)
    if len(popular_orgs_and_persons) == 20:
        break

0 u.s. GPE 19
2 en-ger-land GPE 2
3 atÂ ORGANIZATION 2
4 u.s GPE 2
5 thatÂ ORGANIZATION 2
6 kÃ² ORGANIZATION 2
7 euro2016Â ORGANIZATION 2
8 rainingÂ ORGANIZATION 2
9 rainÂ ORGANIZATION 2
10 weatherÂ ORGANIZATION 2
11 lightningÂ ORGANIZATION 2
12 l.a. GPE 2
13 felicitÃ ORGANIZATION 2
14 zÃ¼rich ORGANIZATION 2
15 fÃªte ORGANIZATION 2
16 brisk_and_vagabond-eyeopener_2007__enviromental_product- GPE 1
17 nycÂ ORGANIZATION 1
18 travelÂ ORGANIZATION 1
20 brisk_and_vinylgroover-rain_fall_down__checkin_da_cutz- GPE 1
21 whiteaddictÂ ORGANIZATION 1
22 inc. ORGANIZATION 1
23 perÃº ORGANIZATION 1
24 brisk_and_ham-on_and_on_ GPE 1
25 sÃ³ria ORGANIZATION 1
26 ____________________________________ ORGANIZATION 1
27 firstÂ ORGANIZATION 1
28 moanaÂ ORGANIZATION 1
29 brisk_and_vagabond_vs_uraken-tokyo_rush__back_in_town- GPE 1
30 ludik.hugo ORGANIZATION 1
31 littleÂ ORGANIZATION 1
32 â¦and GPE 1
33 brÃ¼ssel ORGANIZATION 1
34 brisk_and_ham-gettin_high__serious_hardcore- GPE 1
35 kohlisoonÂ ORGANIZATION 1

In [23]:
import spacy
from spacy import displacy
nlp = spacy.load("en")

In [24]:
cntr = Counter()
ents_dict = {}

In [25]:
def parse_string_entities(text_string, cntr, ents_dict):
    doc = nlp(text_string)
    ents = [(e.text, e.label_) for e in doc.ents]
    for entity in ents:
#         print("text:", entity[0])
#         print("label:", entity[1])
        cntr[entity[0]] += 1
        if entity[0] not in ents_dict:
            ents_dict[entity[0]] = entity[1]
        else:
            if ents_dict[entity[0]] != entity[1]:
                print("ENTITIES MISMATCH:\n\tentity text: %s\n\tdict label: %s\n\tnew label: %s\n" % (entity[0], ents_dict[entity[0]], entity[1]))
                

In [26]:
# !%%time
for i in range(len(combine_df)):
#     parse_string_entities(combine_df.iloc[i]['tweet'], cntr, ents_dict)
    parse_string_entities(combine_df.iloc[i]['processed_tweet'], cntr, ents_dict)

ENTITIES MISMATCH:
	entity text: rip
	dict label: ORG
	new label: PERSON

ENTITIES MISMATCH:
	entity text: kinda
	dict label: PERSON
	new label: GPE

ENTITIES MISMATCH:
	entity text: soo
	dict label: NORP
	new label: PERSON

ENTITIES MISMATCH:
	entity text: eur usd
	dict label: ORG
	new label: PERSON

ENTITIES MISMATCH:
	entity text: soo
	dict label: NORP
	new label: ORG

ENTITIES MISMATCH:
	entity text: tbt
	dict label: PERSON
	new label: ORG

ENTITIES MISMATCH:
	entity text: eur usd
	dict label: ORG
	new label: PERSON

ENTITIES MISMATCH:
	entity text: selfie
	dict label: GPE
	new label: PERSON

ENTITIES MISMATCH:
	entity text: euro england
	dict label: WORK_OF_ART
	new label: ORG

ENTITIES MISMATCH:
	entity text: tbt
	dict label: PERSON
	new label: ORG

ENTITIES MISMATCH:
	entity text: tbt
	dict label: PERSON
	new label: ORG

ENTITIES MISMATCH:
	entity text: selfie
	dict label: GPE
	new label: PERSON

ENTITIES MISMATCH:
	entity text: selfie
	dict label: GPE
	new label: PERSON

ENTITI

In [27]:
cntr.most_common(20)

[('today', 1569),
 ('bihday', 1071),
 ('friday', 648),
 ('tomorrow', 589),
 ('first', 519),
 ('summer', 501),
 ('sunday', 495),
 ('one', 490),
 ('tonight', 456),
 ('morning', 332),
 ('fathersday', 329),
 ('saturday', 308),
 ('monday', 231),
 ('america', 225),
 ('days', 223),
 ('orlando', 212),
 ('two', 182),
 ('london', 182),
 ('june', 175),
 ('thursday', 169)]

In [28]:
for common in cntr.most_common(5):
    word = common[0]
    ent_label = ents_dict[word]
    print(word, ent_label)

today DATE
bihday DATE
friday DATE
tomorrow DATE
first ORDINAL


In [29]:
%%time
tmp_cntr = 0
popular_orgs_and_persons = []
for i, common in enumerate(cntr.most_common(10000)):
    word = common[0]
    count = common[1]
    ent_label = ents_dict[word]
    if ent_label == "PERSON" or ent_label == "ORG":
        popular_orgs_and_persons.append((word, ent_label, count))
    else:
        print(i, word, ent_label, count)
    if len(popular_orgs_and_persons) == 20:
        break

0 today DATE 1569
1 bihday DATE 1071
2 friday DATE 648
3 tomorrow DATE 589
4 first ORDINAL 519
5 summer DATE 501
6 sunday DATE 495
7 one CARDINAL 490
8 tonight TIME 456
9 morning TIME 332
10 fathersday DATE 329
11 saturday DATE 308
12 monday DATE 231
13 america GPE 225
14 days DATE 223
15 orlando GPE 212
16 two CARDINAL 182
17 london GPE 182
18 june DATE 175
19 thursday DATE 169
21 day DATE 163
22 us GPE 148
23 the day DATE 143
24 wednesday DATE 136
25 yesterday DATE 133
26 years DATE 130
27 weekend DATE 130
28 selfie GPE 126
30 night TIME 114
31 this week DATE 110
32 tuesday DATE 109
33 this morning TIME 109
34 next week DATE 107
35 weeks DATE 103
36 last night TIME 99
37 florida GPE 97
38 this weekend DATE 96
39 this year DATE 95
40 uk GPE 94
41 daily DATE 94
42 july DATE 84
43 japan GPE 79
44 india GPE 78
47 muslim NORP 73
48 week DATE 73
49 hours TIME 72
50 evening TIME 72
51 miami GPE 72
53 americans NORP 71
55 american NORP 70
56 half CARDINAL 68
57 year DATE 63
58 the weekend DA

In [30]:
popular_orgs_and_persons

[('tbt', 'PERSON', 169),
 ('aww', 'ORG', 118),
 ('gop', 'ORG', 75),
 ('hu', 'PERSON', 74),
 ('nd', 'ORG', 71),
 ('trump', 'PERSON', 70),
 ('blur sun', 'PERSON', 55),
 ('nba', 'ORG', 50),
 ('don', 'PERSON', 42),
 ('hillary', 'PERSON', 42),
 ('islam', 'ORG', 41),
 ('feminismiscancer feminismisterrorism feminismmuktbharat', 'PERSON', 40),
 ('stas', 'PERSON', 40),
 ('xx', 'PERSON', 38),
 ('allahsoil', 'ORG', 34),
 ('sma', 'ORG', 33),
 ('christina grimmie', 'PERSON', 32),
 ('jo cox', 'PERSON', 30),
 ('house', 'ORG', 30),
 ('ios', 'ORG', 30)]